
## Autograded Notebook (Canvas & CodeGrade)

This notebook will be automatically graded. It is designed to test your answers and award points for the correct answers. Following the instructions for each Task carefully.
Instructions

- **Download** this notebook as you would any other ipynb file 
- **Upload** to Google Colab or work locally (if you have that set-up)
- **Delete** `raise NotImplementedError()`

- **Write** your code in the `# YOUR CODE HERE` space


- **Execute** the Test cells that contain assert statements - these help you check your work (others contain hidden tests that will be checked when you submit through Canvas)

- **Save** your notebook when you are finished
- **Download** as a ipynb file (if working in Colab)
- **Upload** your complete notebook to Canvas (there will be additional instructions in Slack and/or Canvas)



# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
Successfully complete all these objectives to earn full credit. 

**Successful completion is defined as passing all the unit tests in each objective.**  

Each unit test that you pass is 1 point. 

There are 5 total possible points in this sprint challenge. 


There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

____

# Before you submit your notebook you must first

1) Restart your notebook's Kernal

2) Run all cells sequentially, from top to bottom, so that cell numbers are sequential numbers (i.e. 1,2,3,4,5...)
- Easiest way to do this is to click on the **Cell** tab at the top of your notebook and select **Run All** from the drop down menu. 

3) Comment out the cell that generates a pyLDAvis visual in objective 4 (see instructions in that section). 
____

## Note:
I'm getting depreciation errors. I think I know how to fix it, but it involves updating my jupyter software package. I didn't want to risk giving myself any errors on the sprint challenge so I haven't updated it yet, and the warnings remain. 



### Import Data

In [2]:
# Basic Packages
import numpy as np
import pandas as pd
import re

# Modeling
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt


In [2]:
from pprint import pprint

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# NLP Packages
import spacy.cli
spacy.cli.download("en_core_web_sm")
nlp = spacy.load('en_core_web_sm')

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
# Load reviews from URL
data_url = 'https://raw.githubusercontent.com/LambdaSchool/data-science-practice-datasets/main/unit_4/unit1_nlp/review_sample.json'

# Import data into a DataFrame named df
df = pd.read_json(data_url, lines=True)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
df.head()

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [6]:
# Visible Testing
assert isinstance(df, pd.DataFrame), 'df is not a DataFrame. Did you import the data into df?'
assert df.shape[0] == 10000, 'DataFrame df has the wrong number of rows.'

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [7]:
# Optional: Consider using spaCy in your function. The spaCy library can be imported by running this cell.
# A pre-trained model (en_core_web_sm) has been made available to you in the CodeGrade container.
# If you DON'T need use the en_core_web_sm model, you can comment it out below.

import spacy
nlp = spacy.load('en_core_web_sm')

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
# Ensuring text column will be read as a string
df['text']=df['text'].astype('str')

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# Add custom stopwords

nlp.Defaults.stop_words |= {'like', 'go', 'good', 'great', 'come', 'love', 's', 't'}

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
def tokenize(doc, allowed = ['NOUN', 'ADJ', 'VERB', 'ADV']):
    """
    Tokenize & Clean Document Strings
    """
    lemmas = []
    
    # remove anything outside of letters and numbers
    doc = re.sub('[^a-zA-Z]',' ', doc)
    
    # remove extra white space
    doc = re.sub("\s+"," ", doc)
    
    text = nlp(doc)
    
    # return list of tokenize, lemmatized, cleaned strings
    # with no stopwords!
    for token in text: 
        if ((token.is_stop == False) and (token.is_punct == False) and
            (token.pos_ != allowed)):
            lemmas.append(token.lemma_.lower())
    return lemmas

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:11: DeprecationWarning: invalid escape sequence \s
<>:11: DeprecationWarning: invalid escape sequence \s
<ipython-input-10-850701c69527>:11: DeprecationWarning: invalid escape sequence \s
  doc = re.sub("\s+"," ", doc)


In [11]:
'''Testing'''
assert isinstance(tokenize(df.sample(n=1)["text"].iloc[0]), list), "Make sure your tokenizer function accepts a single document and returns a list of tokens!"

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# Making new column with tokenize function applied to text column
df['token'] = df['text'].apply(tokenize)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
# Confirm it worked as expected
# There are som extra spaces in the text that could be cleaned out later
df.head(1)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,business_id,cool,date,funny,review_id,stars,text,useful,user_id,token
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w,"[beware, fake, fake, fake, small, business, lo..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews (i.e. create a doc-term matrix).
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, use `NearestNeighbors` model for this. 

In [14]:
# Make sure column is a string
df['token_str'] = df['token'].astype('str')

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
df['token_str'][:3]

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    ['beware', 'fake', 'fake', 'fake', 'small', 'b...
1    ['come', 'lunch', 'togo', 'service', 'quick', ...
2    ['ve', 'vegas', 'dozen', 'time', 'step', 'foot...
Name: token_str, dtype: object

In [16]:
# Create a vector representation of the reviews 
# Name that doc-term matrix "dtm"

tfidf = TfidfVectorizer(tokenizer=tokenize)

# Grabs word counts per document
fm = tfidf.fit_transform(df['token_str'])


dtm = pd.DataFrame(data = fm.toarray(), columns=tfidf.get_feature_names())

dtm.head()

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,,aa,aaa,aaaahhhs,aaasssk,aab,aamco,aand,aaron,ab,...,zuni,zupas,zur,zuzana,zuzu,zxkxko,zyr,zyrtec,zzaplon,zzzzzzzzz
0,0.022947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.063886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.014944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.049592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.028545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Create and fit a NearestNeighbors model named "nn"
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm);

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
'''Testing.'''
assert nn.__module__ == 'sklearn.neighbors._unsupervised', ' nn is not a NearestNeighbors instance.'
assert nn.n_neighbors == 10, 'nn has the wrong value for n_neighbors'

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
# Create a fake review and find the 10 most similar reviews

fake_review = ["Quiche was delicious. Service was terrible"]

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
df.head(1)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,business_id,cool,date,funny,review_id,stars,text,useful,user_id,token,token_str
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w,"[beware, fake, fake, fake, small, business, lo...","['beware', 'fake', 'fake', 'fake', 'small', 'b..."


In [21]:
df_analysis = df.copy()
fake_review = ["Quiche was delicious. Service was terrible"]

appendable = pd.DataFrame({'business_id': [0], 'cool': [5], 'date': [0], 
                           'funny': [3], 'review_id':[3934], 'stars':[2], 
                           'text':[fake_review],'useful': [3], 'user_id': [0], 
                           'token':[['quiche','delicious', 'service','terrible']]})

df_analysis=df_analysis.append(appendable).reset_index()

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
my_rev = dtm.iloc[1000].values
job = [my_rev]
neigh_dist, neigh_index = nn.kneighbors(job)
index_of_similar = neigh_index[0]

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
# Printing Reviews That Are Similar

for i in range(len(index_of_similar)):
    print(df_analysis.iloc[(index_of_similar[i])]['text'])

I have been a client since November 2013 and I can honestly say the I have seen excellent results with Debi. She takes the time to listen to my concerns and determines the perfect treatment plan for my particular issues. We have even discussed a very frustrating problem of adult acne, for which Debi has suggested great products that have helped. I am very happy with all aspects of the care I receive and plan to continue my treatments here with Debi.
I just wanted to say a HUGE THANK YOU to Debi and her staff at FitForABride!  I was just married on November 8th, 2014 at the Mandalay Bay in Las Vegas.  I decided to go with the "Ivory Sweet Package" which included (1) Steaming of Wedding Gown and veil and (2) Full pick up and deliver to and from hotel bell desk for only $140.  I flew into Las Vegas a few days before the wedding.  The airline I flew in on pretty much smashed and wrinkled my dress in the first class closet no matter what care I took.  Debi was quick to take away all of my f

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a pipeline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier.
    - Use that pipeline to train a model to predict the `stars` feature (i.e. the labels). 
    - Use that Pipeline to predict a star rating for your fake review from Part 2. 



2. Create a parameter dict including `one parameter for the vectorizer` and `one parameter for the model`. 
    - Include 2 possible values for each parameter
    - **Use `n_jobs` = 1** 
    - Due to limited computational resources on CodeGrader `DO NOT INCLUDE ADDITIONAL PARAMETERS OR VALUES PLEASE.`
    
    
3. Train the entire pipeline with a GridSearch
    - Name your GridSearch object as `gs`

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# Name the gridsearch instance "gs"

# Assign X & Y
X = df['token_str']
Y = df['stars']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [1]:
X

NameError: name 'X' is not defined

In [25]:
#Creating Pipeline
tfidf = TfidfVectorizer(stop_words="english")
rfc = RandomForestClassifier()

pipeline = Pipeline([("vectorize", tfidf),
                 ("class", rfc)])

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
# Set Parameters for TFIDF and RFC
parameters = {
    # parameter for the vectorizer
    "vectorize__max_df": [0.75, 1.0], 
    # parameter for the kneighbors model
    "class__n_estimators":[10,20],
}

gs = GridSearchCV(pipeline,
                 parameters, 
                 cv=3,
                 n_jobs= 1,
                 verbose=1)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
gs.fit(X_train, y_train)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vectorize',
                                        TfidfVectorizer(stop_words='english')),
                                       ('class', RandomForestClassifier())]),
             n_jobs=1,
             param_grid={'class__n_estimators': [10, 20],
                         'vectorize__max_df': [0.75, 1.0]},
             verbose=1)

In [28]:
gs.best_score_

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.5510439097325152

In [29]:
y_pred_test = gs.predict(X_test)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
# Actually slightly higher than training data which is interesting

accuracy_score(y_test, y_pred_test)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.5351515151515152

In [31]:
# Best Performing Nearest Neighbors Model for this Data so far
best = gs.best_estimator_

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
gs_predicted = gs.predict(fake_review)
print('Predicted number of stars is:', gs_predicted[0], 'for the review', fake_review)

Predicted number of stars is: 1 for the review ['Quiche was delicious. Service was terrible']


/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
review_2 = ['Terrible service. Disgusting food.']

gs_predict2 = gs.predict(review_2)
print('Predicted number of stars is:', gs_predict2[0], 'for the review', review_2)

Predicted number of stars is: 1 for the review ['Terrible service. Disgusting food.']


/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
# Visible Testing
prediction = gs.predict(["I wish dogs knew how to speak English."])[0]
assert prediction in df.stars.values, 'You gs object should be able to accept raw text within a list. Did you include a vectorizer in your pipeline?'

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Set num_topics to `5`
    - Name your LDA model `lda`
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

When you instantiate your LDA model, it should look like this: 

```python
lda = LdaModel(corpus=corpus,
               id2word=id2word,
               random_state=723812,
               num_topics = num_topics,
               passes=1
              )

```

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

## Note about  pyLDAvis

**pyLDAvis** is the Topic modeling package that we used in class to visualize the topics that LDA generates for us.

You are welcomed to use pyLDAvis if you'd like for your visualization. However, **you MUST comment out the code that imports the package and the cell that generates the visualization before you submit your notebook to CodeGrade.** 

Although you should leave the print out of the visualization for graders to see (i.e. comment out the cell after you run it to create the viz). 

In [35]:
from gensim import corpora
# Due to limited computationalresources on CodeGrader, use the non-multicore version of LDA 
from gensim.models.ldamodel import LdaModel
import gensim
import re

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
df.head(1)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,business_id,cool,date,funny,review_id,stars,text,useful,user_id,token,token_str
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w,"[beware, fake, fake, fake, small, business, lo...","['beware', 'fake', 'fake', 'fake', 'small', 'b..."


### 1. Estimate a LDA topic model of the review tex

In [37]:
# Remember to read the LDA docs for more information on the various class attirbutes and methods available to you
# in the LDA model: https://radimrehurek.com/gensim/models/ldamodel.html

# don't change this value 
num_topics = 5

# Creating library and corpus

# Create Dictionary
id2word = corpora.Dictionary(df['token'])

# Term Document Frequency for each doc -> doc-term matrix 
corpus = [id2word.doc2bow(list_of_token) for list_of_token in df['token']]


/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
lda = LdaModel(corpus=corpus,
               id2word=id2word,
               random_state=723812,
               num_topics = num_topics,
               passes=1
              )
lda.save('model5.gensim')

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
pprint(lda.print_topics())

[(0,
  '0.012*"place" + 0.009*"t" + 0.009*"s" + 0.008*"food" + 0.007*"la" + '
  '0.007*"try" + 0.006*"de" + 0.005*"d" + 0.004*"class" + 0.004*"donut"'),
 (1,
  '0.018*"food" + 0.018*"t" + 0.013*"order" + 0.011*"s" + 0.009*"place" + '
  '0.009*"service" + 0.008*"time" + 0.007*"chicken" + 0.007*"try" + '
  '0.006*"restaurant"'),
 (2,
  '0.018*"t" + 0.017*"s" + 0.010*"time" + 0.010*"place" + 0.008*"service" + '
  '0.008*"go" + 0.006*"don" + 0.005*"food" + 0.005*"get" + 0.005*"work"'),
 (3,
  '0.013*"place" + 0.010*"s" + 0.009*"time" + 0.008*"food" + 0.008*"t" + '
  '0.008*"order" + 0.008*"get" + 0.007*"come" + 0.006*"delicious" + '
  '0.006*"try"'),
 (4,
  '0.016*"place" + 0.010*"friendly" + 0.010*"amazing" + 0.010*"good" + '
  '0.010*"nice" + 0.010*"recommend" + 0.009*"service" + 0.009*"time" + '
  '0.009*"staff" + 0.008*"food"')]


/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Testing

In [40]:
# Visible Testing
assert lda.get_topics().shape[0] == 5, 'Did your model complete its training? Did you set num_topics to 5?'

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
id2word

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
corpus_2 = [[(id2word[i], freq) for i, freq in doc] for doc in corpus[:1]]

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### 2. Create 1-2 visualizations of the results

In [45]:
import seaborn as sns
import matplotlib.pyplot as plt

#Use pyLDAvis (or a ploting tool of your choice) to visualize your results 
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
pyLDAvis.display(vis)

/Users/jenniferquigley/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-gQtURTt4/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### 3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

Interestingly, there seemt to be a few clear topics in the reviews. Topic 3, for example, seems to focus on doctor's office reviews. It includes key words like "dr", "appointment", and "staff." It also did not have much overlap with the other topics. Actually, all of the topics seemed to have mostly distinct keywords and didn't have a lot of overlap.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)